In [45]:
## In-Class Assignment 8 - Evan Callaghan

import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import accuracy_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445-bucket-callaghan'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## 1. Using the pandas library to read the csv data file and create a data-frame called heart
heart = pd.read_csv(file_content_stream)

heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [46]:
## 2. Removing observations with missing values

heart = heart.dropna()

In [47]:
## 3. Using age, currentSmoker, totChol, BMI, and heartRate as the predictor variables, and TenYearCHD is the target variable to split the data into two 
## data-frames: train (80%) and test (20%).

X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [48]:
## 4. Normalizing the inputs variables of the train and test datasets using the L2 normalization.

def l2_normalization(X):
    X_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    l2_norm = (X - X_mean) / l2
    return l2_norm

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)

In [49]:
## 5. Using the train data-frame to build a logistic regression model in which: age, currentSmoker, totChol, BMI, and heartRate are the predictor 
## variables, and TenYearCHD is the target variable. 

md1 = LogisticRegression().fit(X_train, Y_train)

pred1 = md1.predict_proba(X_test)[:, 1]

pred1 = np.where(pred1 < 0.25, 0, 1)

## Reporting the accuracy
md1_accuracy = accuracy_score(Y_test, pred1)

print("Model 1 Accuracy:", md1_accuracy)


Model 1 Accuracy: 0.8592896174863388


In [50]:
## 6.  Using the train data-frame, build a perceptron model in which: age, currentSmoker, totChol, BMI, and heartRate are the predictor variables, 
## and TenYearCHD is the target variable. 

## Using penalty = ’l2’ and alpha = 0.01.

md2 = Perceptron(penalty = 'l2', alpha = 0.01).fit(X_train, Y_train)

## To see the optimal weights:
##md2.coef_

## Predicting on the test set
pred2 = md2.predict(X_test)

## Reporting the accuracy
md2_accuracy = accuracy_score(Y_test, pred2)

print("Model 2 Accuracy:", md2_accuracy)

Model 2 Accuracy: 0.1407103825136612


In [51]:
## 7. Using the results from parts (5) and (6), we would use the logistic regression model to predict TenYearCHD because it has the higher accuracy score.